In [1]:
import dace
import cupy as cp

from IPython.display import Code
from dace.config import Config


print(Config.get('compiler', 'cuda', 'implementation'))


experimental


In [2]:
bs = 512
ns = 1024
BS = dace.symbol('BS')
NS = dace.symbol('NS')

START = dace.symbol('START')
WS = dace.symbol('WS')
STRIDE = dace.symbol('STRIDE')

start = 2
stride = 3
ws = 16
@dace.program
def symbolic_warp_map(A: dace.uint32[NS] @ dace.dtypes.StorageType.GPU_Global, B: dace.uint32[NS] @ dace.dtypes.StorageType.GPU_Global):
    """
    Focus is in the use of symbolic variables in the MAP.
    """
    A[:] = B[:]

sdfg = symbolic_warp_map.to_sdfg()

In [3]:
Code(sdfg.generate_code()[0].clean_code, language='cpp')

not_in_kernel_code: True
is_between_access_nodes: True
involves_gpu_or_pinned: True


copy_context.src_storage: StorageType.GPU_Global
copy_context.dst_storage: StorageType.GPU_Global
is_not_cpu_to_cpu: True


/* DaCe AUTO-GENERATED FILE. DO NOT MODIFY */
#include <dace/dace.h>
#include "../../include/hash.h"

struct symbolic_warp_map_state_t {
    dace::cuda::Context *gpu_context;
};

void __program_symbolic_warp_map_internal(symbolic_warp_map_state_t*__state, dace::uint * __restrict__ A, dace::uint * __restrict__ B, int NS)
{

    {

        DACE_GPU_CHECK(cudaMemcpyAsync(A, B, NS * sizeof(dace::uint), cudaMemcpyDeviceToDevice, __state->gpu_context->streams[0]));

    }
}

DACE_EXPORTED void __program_symbolic_warp_map(symbolic_warp_map_state_t *__state, dace::uint * __restrict__ A, dace::uint * __restrict__ B, int NS)
{
    __program_symbolic_warp_map_internal(__state, A, B, NS);
}
DACE_EXPORTED int __dace_init_experimental_cuda(symbolic_warp_map_state_t *__state, int NS);
DACE_EXPORTED int __dace_exit_experimental_cuda(symbolic_warp_map_state_t *__state);

DACE_EXPORTED symbolic_warp_map_state_t *__dace_init_symbolic_warp_map(int NS)
{
    int __result = 0;
    symbolic_warp_map_state_t *__state = new symbolic_warp_map_state_t;


    __result |= __dace_init_experimental_cuda(__state, NS);

    if (__result) {
        delete __state;
        return nullptr;
    }
    return __state;
}

DACE_EXPORTED int __dace_exit_symbolic_warp_map(symbolic_warp_map_state_t *__state)
{
    int __err = 0;

    int __err_experimental_cuda = __dace_exit_experimental_cuda(__state);
    if (__err_experimental_cuda) {
        __err = __err_experimental_cuda;
    }
    delete __state;
    return __err;
}

In [4]:
Code(sdfg.generate_code()[1].clean_code, language='cpp')

not_in_kernel_code: True
is_between_access_nodes: True
involves_gpu_or_pinned: True


copy_context.src_storage: StorageType.GPU_Global
copy_context.dst_storage: StorageType.GPU_Global
is_not_cpu_to_cpu: True


#include <cuda_runtime.h>
#include <dace/dace.h>


struct symbolic_warp_map_state_t {
    dace::cuda::Context *gpu_context;
};



DACE_EXPORTED int __dace_init_experimental_cuda(symbolic_warp_map_state_t *__state, int NS);
DACE_EXPORTED int __dace_exit_experimental_cuda(symbolic_warp_map_state_t *__state);



int __dace_init_experimental_cuda(symbolic_warp_map_state_t *__state, int NS) {
    int count;

    // Check that we are able to run cuda code
    if (cudaGetDeviceCount(&count) != cudaSuccess)
    {
        printf("ERROR: GPU drivers are not configured or cuda-capable device "
               "not found\n");
        return 1;
    }
    if (count == 0)
    {
        printf("ERROR: No cuda-capable devices found\n");
        return 2;
    }

    // Initialize cuda before we run the application
    float *dev_X;
    DACE_GPU_CHECK(cudaMalloc((void **) &dev_X, 1));
    DACE_GPU_CHECK(cudaFree(dev_X));

    

    __state->gpu_context = new dace::cuda::Context(1, 1);

    // Create cuda streams and events
    for(int i = 0; i < 1; ++i) {
        DACE_GPU_CHECK(cudaStreamCreateWithFlags(&__state->gpu_context->internal_streams[i], cudaStreamNonBlocking));
        __state->gpu_context->streams[i] = __state->gpu_context->internal_streams[i]; // Allow for externals to modify streams
    }
    for(int i = 0; i < 1; ++i) {
        DACE_GPU_CHECK(cudaEventCreateWithFlags(&__state->gpu_context->events[i], cudaEventDisableTiming));
    }

    

    return 0;
}

int __dace_exit_experimental_cuda(symbolic_warp_map_state_t *__state) {
    

    // Synchronize and check for CUDA errors
    int __err = static_cast<int>(__state->gpu_context->lasterror);
    if (__err == 0)
        __err = static_cast<int>(cudaDeviceSynchronize());

    // Destroy cuda streams and events
    for(int i = 0; i < 1; ++i) {
        DACE_GPU_CHECK(cudaStreamDestroy(__state->gpu_context->internal_streams[i]));
    }
    for(int i = 0; i < 1; ++i) {
        DACE_GPU_CHECK(cudaEventDestroy(__state->gpu_context->events[i]));
    }

    delete __state->gpu_context;
    return __err;
}

DACE_EXPORTED bool __dace_gpu_set_stream(symbolic_warp_map_state_t *__state, int streamid, gpuStream_t stream)
{
    if (streamid < 0 || streamid >= 1)
        return false;

    __state->gpu_context->streams[streamid] = stream;

    return true;
}

DACE_EXPORTED void __dace_gpu_set_all_streams(symbolic_warp_map_state_t *__state, gpuStream_t stream)
{
    for (int i = 0; i < 1; ++i)
        __state->gpu_context->streams[i] = stream;
}